In [ ]:
import requests
from datetime import datetime, timedelta
import oracledb
import json
import time

In [ ]:
def get_krx_stock_data(date):
    url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"
    headers = {
        'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }
    params = {
        'bld': 'dbms/MDC/STAT/standard/MDCSTAT01501',
        'locale': 'ko_KR',
        'mktId': 'ALL',
        'trdDd': date,
        'share': '1',
        'money': '1',
        'csvxls_isNo': 'false',
    }
    
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    
    if 'OutBlock_1' not in data or not data['OutBlock_1']:
        print(f"No data available for {date}")
        return None
    
    print(f"Number of items in OutBlock_1: {len(data['OutBlock_1'])}")
    return data['OutBlock_1']

In [ ]:
def save_to_oracle(connection, data, date):
    cursor = connection.cursor()
    
    insert_sql = """
    INSERT INTO STOCK (record_date, stock_code, name, stock_type, closing_price, opening_price, high_price, low_price)
    VALUES (:1, :2, :3, :4, :5, :6, :7, :8)
    """
    
    records = []
    for item in data:
        try:
            record = (
                datetime.strptime(date, '%Y%m%d').date(),
                item.get('ISU_SRT_CD', ''),
                item.get('ISU_ABBRV', ''),
                item.get('MKT_NM', ''),
                float(item.get('TDD_CLSPRC', '0').replace(',', '')),
                float(item.get('TDD_OPNPRC', '0').replace(',', '')),
                float(item.get('TDD_HGPRC', '0').replace(',', '')),
                float(item.get('TDD_LWPRC', '0').replace(',', ''))
            )
            records.append(record)
        except (ValueError, KeyError) as e:
            print(f"Error processing item: {e}")
            print(f"Problematic item: {item}")
    
    if records:
        cursor.executemany(insert_sql, records)
        connection.commit()
        print(f"Data saved for {date}. Records: {len(records)}")
    else:
        print(f"No valid records to insert for {date}")

In [ ]:
def get_and_save_data_for_date_range(connection, start_date, end_date):
    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() < 5:  # 0-4는 월-금
            date_str = current_date.strftime("%Y%m%d")
            print(f"Fetching data for {date_str}")
            data = get_krx_stock_data(date_str)
            if data:
                save_to_oracle(connection, data, date_str)
            else:
                print(f"No data available for {date_str}")
            
            time.sleep(1)  # 1초 대기하여 API 요청 속도 제한
        
        current_date += timedelta(days=1)

# 실행
try:
    connection = oracledb.connect(user="final", password='final1234', dsn="localhost:1521/XE")
    
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=730)  # 약 2년 전
    
    print(f"Fetching data from {start_date} to {end_date}")
    get_and_save_data_for_date_range(connection, start_date, end_date)

    # 저장된 데이터 확인
#     cursor = connection.cursor()
#     cursor.execute("SELECT COUNT(*) FROM STOCK")
#     count = cursor.fetchone()[0]
#     print(f"Total records in database: {count}")

#     query = "SELECT MIN(record_date), MAX(record_date) FROM STOCK"
#     cursor.execute(query)
#     min_date, max_date = cursor.fetchone()
#     print(f"Date range in database: from {min_date} to {max_date}")
except oracledb.DatabaseError as e:
    error, = e.args
    print(f"Database error occurred: {error.code} - {error.message}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    import traceback
    print(traceback.format_exc())
finally:
    if 'connection' in locals() and connection:
        connection.close()